In [1]:
use mavenfuzzyfactory

Commands completed successfully

Total execution time: 00:00:00

**Subiect: Sales Trends**

<span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">We're about to launch a new product, and I'd like to do a&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">deep dive on our current flagship product.&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Can you please pull monthly trends to date for number of&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">sales, total revenue, and total margin generated for the&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">business?</span>

In [2]:
show tables

(6 row(s) affected)

Total execution time: 00:00:01.010

Tables_in_mavenfuzzyfactory
order_item_refunds
order_items
orders
products
website_pageviews
website_sessions


In [3]:
select * from orders limit 3

(3 row(s) affected)

Total execution time: 00:00:01.012

order_id,created_at,website_session_id,user_id,primary_product_id,items_purchased,price_usd,cogs_usd
1,2012-03-19 10:42:46,20,20,1,1,49.99,19.49
2,2012-03-19 19:27:37,104,104,1,1,49.99,19.49
3,2012-03-20 06:44:45,147,147,1,1,49.99,19.49


In [6]:
select YEAR(date(created_at)) as yr, MONTH(DATE(created_at)) as mo,
count(order_id) as sales, sum(price_usd) as revenue, 
sum(price_usd - cogs_usd) as margin
from orders
where created_at < '2013-01-04'
GROUP by 1,2

(11 row(s) affected)

Total execution time: 00:00:01.026

yr,mo,sales,revenue,margin
2012,3,60,2999.40,1830.00
2012,4,99,4949.01,3019.50
2012,5,108,5398.92,3294.00
2012,6,140,6998.60,4270.00
2012,7,169,8448.31,5154.50
2012,8,228,11397.72,6954.00
2012,9,287,14347.13,8753.50
2012,10,371,18546.29,11315.50
2012,11,618,30893.82,18849.00
2012,12,506,25294.94,15433.00
